In [139]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
from selenium import webdriver
import pymysql
import re

db = pymysql.connect(host='52.78.23.232', user='root', password='readers7', port=3306, db='readers', charset='utf8')
# browser = webdriver.Chrome('C:/Users/KSY/Documents/GitHub/Readers/crawling/chromedriver.exe')
cursor = db.cursor()
sql = "insert into toon_info(toon_name, toon_url, description, toon_site, toon_weekday, wrt_nm, toon_genre, is_end)\
       values (%s, %s, %s, %s, %s, %s, %s, %s)"

#웹 페이지를 열고 소스코드를 읽어오는 작업
def readHtml(url):
    # print(url)
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    html.close()
    return soup

soup = readHtml("http://comic.naver.com/webtoon/weekday.nhn")

#요일별 웹툰영역 추출하기
data1_list=soup.findAll('div',{'class':'thumb'})
# pprint(data1_list)

#전체 웹툰 리스트
a_list = []
for data1 in data1_list:
    a_list.extend(data1.findAll('a'))
# pprint(a_list)

# 모든 요일 링크 추출
for a in a_list:
    src = a['href']
    url = 'https://comic.naver.com' + src + '&page=' # 웹툰 페이지 추출
    
    soup = readHtml(url+'1') # 네이버 웹툰 첫페이지
    
    # 요일 추출
    if 'weekday=mon' in url:
        weekday = 'mon'
    if 'weekday=tue' in url:
        weekday = 'tue'
    if 'weekday=wed' in url:
        weekday = 'wed'
    if 'weekday=thu' in url:
        weekday = 'thu'
    if 'weekday=fri' in url:
        weekday = 'fri'
    if 'weekday=sat' in url:
        weekday = 'sat'
    if 'weekday=sun' in url:
        weekday = 'sun'
    
    detail = soup.find('div', {'class':'detail'})
    title = detail.find('h2')
    
    wrt_nm = str(title.find('span', {'class':'wrt_nm'}))
    wrt_nm = re.sub('<.+?>', '', wrt_nm, 0).strip() # 작가 추출
    
    title = str(title).replace('\t', '').replace('컷툰', '')
    title = re.sub('<.+?>', '', title, 0).strip()
    terminator = title.index('\n')
    title = title[:terminator].strip('컷') # 제목 추출
    
    genre = str(detail.find('span', {'class':'genre'}))
    genre = re.sub('<.+?>', '', genre, 0).strip() # 장르 추출

    desc = str(detail.find('p')).replace('<br/>', '\n')
    desc = re.sub('<.+?>', '', desc, 0).strip() # 설명 추출
    
    thumb = soup.find('img')['src'] # 썸네일 주소 추출
    
    cursor.execute(sql, (title, url, desc, 'naver', weekday, wrt_nm, genre, 'X'))
    db.commit()

In [140]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
from selenium import webdriver
import pymysql
import re

db = pymysql.connect(host='52.78.23.232', user='root', password='readers7', port=3306, db='readers', charset='utf8')
# browser = webdriver.Chrome('C:/Users/KSY/Documents/GitHub/Readers/crawling/chromedriver.exe')
cursor = db.cursor()
sql = "insert into toon_info(toon_name, toon_url, description, toon_site, toon_weekday, wrt_nm, toon_genre, is_end)\
       values (%s, %s, %s, %s, %s, %s, %s, %s)"

#웹 페이지를 열고 소스코드를 읽어오는 작업
def readHtml(url):
    # print(url)
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    html.close()
    return soup

soup = readHtml("https://comic.naver.com/webtoon/finish.nhn")

#요일별 웹툰영역 추출하기
data1_list=soup.findAll('div',{'class':'thumb'})
# pprint(data1_list)

#전체 웹툰 리스트
a_list = []
for data1 in data1_list:
    a_list.extend(data1.findAll('a'))
# pprint(a_list)

# 웹툰 정보 추출
for a in a_list:
    src = a['href']
    url = 'https://comic.naver.com' + src + '&page=' # 웹툰 페이지 추출
    
    soup = readHtml(url+'1') # 네이버 웹툰 첫페이지
    
    detail = soup.find('div', {'class':'detail'})
    title = detail.find('h2')
    
    wrt_nm = str(title.find('span', {'class':'wrt_nm'}))
    wrt_nm = re.sub('<.+?>', '', wrt_nm, 0).strip() # 작가 추출
    
    title = str(title).replace('\t', '').replace('컷툰', '')
    title = re.sub('<.+?>', '', title, 0).strip()
    terminator = title.index('\n')
    title = title[:terminator].strip('컷') # 제목 추출
    
    genre = str(detail.find('span', {'class':'genre'}))
    genre = re.sub('<.+?>', '', genre, 0).strip() # 장르 추출

    desc = str(detail.find('p')).replace('<br/>', '\n')
    desc = re.sub('<.+?>', '', desc, 0).strip() # 설명 추출
    
    thumb = soup.find('img')['src'] # 썸네일 주소 추출
    
    cursor.execute(sql, (title, url, desc, 'naver', None, wrt_nm, genre, 'O'))
    db.commit()